In [2]:
import imp
import torch
import PIL.Image
import os
import torchvision
import numpy as np
import random

# globals

MainModel = imp.load_source('MainModel', '/home/kylesargent/resnet50_128_pytorch/resnet50_128_pytorch.py') 
model = torch.load('/home/kylesargent/resnet50_128_pytorch/resnet50_128_pytorch.pth')
model.cuda()

DATASET_PATH = '/home/kylesargent/test/'

batch_size = 16
alpha = .01

/home/kylesargent/.local/lib/python2.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/kylesargent/.local/lib/python2.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [3]:
import load_and_transform
reload(load_and_transform)
from load_and_transform import transform_image, get_image_list, load_image_batch

In [4]:
# data preprocessing

import random

images = get_image_list()
np.random.shuffle(images)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(images[:, 1])

labels = le.transform(images[:, 1])

len(labels)

169396

In [5]:
import torch.nn as nn
from torch.autograd import Variable
cluster_model = nn.Sequential(nn.Linear(128, 128), nn.ReLU(), nn.Linear(128, 128)).cuda()

In [6]:
import torch
from torch.autograd import Variable
torch.cuda.memory_allocated()

95459840L

In [10]:
# training loop
batch_size = 64
alpha = .0001
lr = .001
train_frac = .8

from tqdm import tqdm

torch.cuda.empty_cache()
torch.cuda.memory_allocated()

from torch.optim import Adam


optimizer = Adam(cluster_model.parameters(), lr=lr)

for i in tqdm(range(len(images) * train_frac // batch_size)):

    batch_labels = torch.from_numpy(labels[i:i + batch_size])
    batch_images = load_image_batch(images[:, 0][i:i + batch_size])

    with torch.no_grad():
        batch_features = model(batch_images)

    batch_features = Variable(batch_features.squeeze())
    batch_codes = cluster_model(batch_features)
    
    M = pairwise_distances(batch_codes, batch_codes)
    
    # delta[i][j] is 1 when code i and code j are the same person
    delta = (batch_labels.unsqueeze(0) - batch_labels.unsqueeze(1)) == 0

    discounts = (delta - alpha).type('torch.FloatTensor').cuda()
    loss = torch.sum(discounts*M)
        
    print(loss)
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



  0%|          | 1/2646 [00:00<38:50,  1.13it/s]

tensor(80.0015, device='cuda:0', grad_fn=<SumBackward0>)



  0%|          | 2/2646 [00:01<36:47,  1.20it/s]

tensor(166.4733, device='cuda:0', grad_fn=<SumBackward0>)



  0%|          | 3/2646 [00:02<35:23,  1.24it/s]

...


  1%|          | 15/2646 [00:11<32:37,  1.34it/s]

tensor(1.3769, device='cuda:0', grad_fn=<SumBackward0>)



  1%|          | 16/2646 [00:11<32:34,  1.35it/s]

tensor(5.8798, device='cuda:0', grad_fn=<SumBackward0>)


KeyboardInterrupt: 

In [51]:
# chinese restaurant process

from scipy.stats import multivariate_normal

cluster_ids = []
cluster_codes = []
sigma = .1
theta = 1

batch_size = 1

for i in (range(int(len(images) * .8), 169396)):
    
    
    batch_labels = torch.from_numpy(labels[i:i + batch_size])
    batch_images = load_image_batch(images[:, 0][i:i + batch_size])

    with torch.no_grad():
        batch_features = model(batch_images)

    batch_features = Variable(batch_features.squeeze())
    batch_code = cluster_model(batch_features).data.numpy()
    
    k = i - int(len(images) * .8)
    if k == 0:
        # set up first cluster
        cluster_ids = [[0]]
        cluster_codes = [[batch_code]]
        cluster_centers = [batch_code]
    else:
        cluster_priors = map(lambda ks: float(len(ks))/(k + theta), cluster_ids)
        cluster_priors += [float(k)/(k + theta)]
        
        cluster_likelihoods = [
            multivariate_normal.pdf(
                batch_code, mean=cluster_center, cov=np.eye(128)*sigma
            ) 
            for cluster_center in cluster_centers
        ]
        
        
        # assert(len(cluster_priors) == len(cluster_ids))
        
        
        
    _ = input()

1


AssertionError: 

In [26]:
x = torch.randn(64, 128)

torch.mm(x, x.transpose(0, 1)))

torch.Size([64, 64])

In [12]:
x = [[1,2,3], [4,5,6]]
x[0] += [4]

x

[[1, 2, 3, 4], [4, 5, 6]]

In [33]:
np.random.multinomial(1, [.2, .8])

array([0, 1])

In [8]:
def pairwise_distances(x,y):
    x_norms = torch.sum(x*x, dim=1).unsqueeze(1)
    y_norms = torch.sum(y*y, dim=1).unsqueeze(0)    
    return x_norms - 2*torch.mm(x, y.transpose(0, 1)) + y_norms